In [80]:
import requests
import pandas as pd
import numpy as np
import openpyxl
import os 

# Read in list of assets of interest and convert to list
assetFrame = pd.read_excel('assetList2.xlsx')
assetList = assetFrame['Asset'].tolist()
print('Asset list successfully imported')

# Initialize an empty list to store the data
data_list = []

for asset in assetList:

    # Initial URL for the first API call
    base_url = "https://clinicaltrials.gov/api/v2/studies"
    params = {
        "query.term": asset,
        "pageSize": 100
    }
    # Assign asset name from parameter
    assetName = params['query.term']

    # Loop until there is no nextPageToken
    while True:
        # Print the current URL (for debugging purposes); Access by looping parameters
        print("Fetching data from:", base_url + '?' + '&'.join([f"{k}={v}" for k, v in params.items()]))
        
        # Send a GET request to the API
        response = requests.get(base_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            studies = data.get('studies', [])  # Extract the list of studies, pass empty parameters

            # Loop through each study and extract specific information
            for study in studies:
                # Safely access nested keys, Indicates Unknown if unavailable 
                nctId = study['protocolSection']['identificationModule'].get('nctId', 'Unknown')
                studyTitle = study['protocolSection']['identificationModule'].get('officialTitle', 'Unknown')
                sponsor = study['protocolSection']['sponsorCollaboratorsModule'].get('leadSponsor', {}).get('name', 'Unknonw Lead Sponsor')
                overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'Unknown')
                enrollment = study['protocolSection']['designModule'].get('enrollmentInfo',{}).get('count', 'Unknown')
                startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', 'Unknown Date')
                conditions = ', '.join(study['protocolSection']['conditionsModule'].get('conditions', ['No conditions listed']))
                acronym = study['protocolSection']['identificationModule'].get('acronym', 'Unknown')

                # Extract interventions safely
                interventions_list = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
                interventions = ', '.join([intervention.get('name', 'No intervention name listed') for intervention in interventions_list]) if interventions_list else "No interventions listed"

                # Extract outcomes safely
                primaryOutcomes_list = study['protocolSection'].get('outcomesModule', {}).get('primaryOutcomes', [])
                primaryOutcomes = ', '.join([measure.get('measure', 'No primary outcome indicated') for measure in primaryOutcomes_list]) if primaryOutcomes_list else "No primary outcomes listed"
                secondaryOutcomes_list = study['protocolSection'].get('outcomesModule', {}).get('secondaryOutcomes', [])
                secondaryOutcomes = ', '.join([measure.get('measure', 'No secomdary outcomes indicated') for measure in secondaryOutcomes_list]) if secondaryOutcomes_list else "No secondary outcomes listed"
                
                # Extract locations safely
                locations_list = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
                locations = ', '.join([f"{location.get('city', 'No City')} - {location.get('country', 'No Country')}" for location in locations_list]) if locations_list else "No locations listed"
                
                # Extract dates and phases
                primaryCompletionDate = study['protocolSection']['statusModule'].get('primaryCompletionDateStruct', {}).get('date', 'Unknown Date')
                studyFirstPostDate = study['protocolSection']['statusModule'].get('studyFirstPostDateStruct', {}).get('date', 'Unknown Date')
                lastUpdatePostDate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', 'Unknown Date')
                studyType = study['protocolSection']['designModule'].get('studyType', 'Unknown')
                phases = ', '.join(study['protocolSection']['designModule'].get('phases', ['Not Available']))

                # Append the data to the list as a dictionary
                data_list.append({
                    "NCT ID": nctId,
                    "Asset": assetName,
                    "Title": studyTitle,
                    "Lead Sponsor": sponsor,
                    "Acronym": acronym,
                    "Overall Status": overallStatus,
                    "Enrollment": enrollment,
                    "Start Date": startDate,
                    "Conditions": conditions,
                    "Interventions": interventions,
                    "Primary Outcome(s)": primaryOutcomes,
                    "Secondary Outcome(s)": secondaryOutcomes,
                    "Locations": locations,
                    "Primary Completion Date": primaryCompletionDate,
                    "Study First Post Date": studyFirstPostDate,
                    "Last Update Post Date": lastUpdatePostDate,
                    "Study Type": studyType,
                    "Phases": phases
                })

            # Check for nextPageToken and update the params or break the loop
            nextPageToken = data.get('nextPageToken')
            if nextPageToken:
                params['pageToken'] = nextPageToken  # Set the pageToken for the next request
            else:
                break  # Exit the loop if no nextPageToken is present
        else:
            print("Failed to fetch data. Status code:", response.status_code)
            break

# Create a DataFrame from the list of dictionaries
currentDataFrame = pd.DataFrame(data_list)

# Identify whether comparator trials excel exists
workingDirectory = os.getcwd()
isExisting = os.path.exists(workingDirectory+'/oldClinicalTrialsData.csv')

# If comparator trial exists already make comparator data frame, otherwise initialize excel
if isExisting == True:

    # Creating old data frame from existing excel
    oldDataDirectory = workingDirectory+'/oldClinicalTrialsData.csv'
    oldDataFrame = pd.read_csv(oldDataDirectory)
    print('Historic clinical trials data recognized and reconciling changes from historic file and new trial updates.')

    # Concatenate old and new data frames based on NCT ID index and swap to be side by side
    concatenateFrames = pd.concat([oldDataFrame.set_index('NCT ID'), currentDataFrame.set_index('NCT ID')], 
            axis='columns', keys=['Old CT Data', 'New CT Data'])
    easyCompareFrames = concatenateFrames.swaplevel(axis='columns')[oldDataFrame.columns[1:]]
    display(easyCompareFrames)

    # Index old and new dataframes on NCT ID
    oldDataFrame.set_index('NCT ID', inplace = True)
    currentDataFrame.set_index('NCT ID', inplace = True)

    # Create dataframes for rows that were added or deleted 
    addedRows = currentDataFrame[~currentDataFrame.index.isin(oldDataFrame.index)]
    deletedRows = oldDataFrame[~oldDataFrame.index.isin(currentDataFrame.index)]
    
    # Find modified rows
    common_index = oldDataFrame.index.intersection(currentDataFrame.index)
    modified_rows = oldDataFrame.loc[common_index] != currentDataFrame.loc[common_index]
    modified_rows = modified_rows[modified_rows.any(axis=1)]
    modified_rows = currentDataFrame.loc[modified_rows.index]

    # Display results
    print("Added rows:\n")
    display(addedRows)
    print("\nDeleted rows:\n")
    display(deletedRows)
    print("\nModified rows:\n")
    display(modified_rows) #Need to fix modified rows

    # For assets that are in both frames, identify when new NCTs have been added, NCTs have changed, and deleted and highlight 
    # Flag trials that are associated from new asset adds and remove
    # Identify where enrollment, start, completion date change

else:

    # Alerting user that no historic file was found
    print('No historic clinical trials data is recognized. We are iniitializing a new file for tracking.')

    # Saving new data frame as a csv
    currentDataFrame.to_csv("oldClinicalTrialsData.csv", index=False)

    # Print the DataFrame
    display(currentDataFrame)
    

Asset list successfully imported
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=Bitopertin&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=GSBR-1290&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=RP-1&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=Ritlecitinib&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=MK-0616&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.term=Retatrutide&pageSize=100
Historic clinical trials data recognized and reconciling changes from historic file and new trial updates.


Asset               \
            Old CT Data  New CT Data   
NCT ID                                 
NCT01636492  Bitopertin   Bitopertin   
NCT05308472  Bitopertin   Bitopertin   
NCT01192880  Bitopertin   Bitopertin   
NCT05883748  Bitopertin   Bitopertin   
NCT02019290  Bitopertin   Bitopertin   
...                 ...          ...   
NCT06297603         NaN  Retatrutide   
NCT06383390         NaN  Retatrutide   
NCT05929079         NaN  Retatrutide   
NCT05882045         NaN  Retatrutide   
NCT05936151         NaN  Retatrutide   

                                                         Title  \
                                                   Old CT Data   
NCT ID                                                           
NCT01636492  Single Oral Ascending Dose Study of Safety, To...   
NCT05308472  (AURORA) A Randomized, Double-blind, Placebo-C...   
NCT01192880  A Phase III, Multi-Center, Randomized, 24 Week...   
NCT05883748  An Open-Label, Long-Term Study to Investigate ...   
NCT02019290  A SINGLE CENTER, OPEN LABEL, FIXED SEQUENCE ST...   
...                                                        ...   
NCT06297603                                                NaN   
NCT06383390                                                NaN   
NCT05929079                                                NaN   
NCT05882045                                                NaN   
NCT05936151                                                NaN   

                                                                \
                                                   New CT Data   
NCT ID                                                           
NCT01636492  Single Oral Ascending Dose Study of Safety, To...   
NCT05308472  (AURORA) A Randomized, Double-blind, Placebo-C...   
NCT01192880  A Phase III, Multi-Center, Randomized, 24 Week...   
NCT05883748  An Open-Label, Long-Term Study to Investigate ...   
NCT02019290  A SINGLE CENTER, OPEN LABEL, FIXED SEQUENCE ST...   
...                                                        ...   
NCT06297603  A Phase 3, Randomized, Multicenter, Double-Bli...   
NCT06383390  A Phase 3, Randomized, Double-Blind, Placebo-C...   
NCT05929079  A Master Protocol to Investigate the Efficacy ...   
NCT05882045  A Randomized, Double-Blind, Phase 3 Study to I...   
NCT05936151  A Phase 2b, Double-Blind Study to Investigate ...   

                   Lead Sponsor                            Acronym  \
                    Old CT Data            New CT Data Old CT Data   
NCT ID                                                               
NCT01636492   Hoffmann-La Roche      Hoffmann-La Roche     Unknown   
NCT05308472  Disc Medicine, Inc     Disc Medicine, Inc     Unknown   
NCT01192880   Hoffmann-La Roche      Hoffmann-La Roche     Unknown   
NCT05883748  Disc Medicine, Inc     Disc Medicine, Inc     Unknown   
NCT02019290   Hoffmann-La Roche      Hoffmann-La Roche     Unknown   
...                         ...                    ...         ...   
NCT06297603                 NaN  Eli Lilly and Company         NaN   
NCT06383390                 NaN  Eli Lilly and Company         NaN   
NCT05929079                 NaN  Eli Lilly and Company         NaN   
NCT05882045                 NaN  Eli Lilly and Company         NaN   
NCT05936151                 NaN  Eli Lilly and Company         NaN   

                                  Overall Status                           \
            New CT Data              Old CT Data              New CT Data   
NCT ID                                                                      
NCT01636492     Unknown                COMPLETED                COMPLETED   
NCT05308472     Unknown    ACTIVE_NOT_RECRUITING    ACTIVE_NOT_RECRUITING   
NCT01192880     Unknown                COMPLETED                COMPLETED   
NCT05883748     Unknown  ENROLLING_BY_INVITATION  ENROLLING_BY_INVITATION   
NCT02019290     Unknown                WITHDRAWN                WITHDRAWN   
...       

Added rows:



,Asset,Title,Lead Sponsor,Acronym,Overall Status,Enrollment,Start Date,Conditions,Interventions,Primary Outcome(s),Secondary Outcome(s),Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases
NCT ID,,,,,,,,,,,,,,,,,
NCT05929066,Retatrutide,A Master Protocol to Investigate the Efficacy ...,Eli Lilly and Company,TRIUMPH-1,RECRUITING,2100,2023-07-10,"Obesity, Overweight, Knee Pain Chronic, Knee O...","Retatrutide, Placebo","Percent Change From Baseline in Body Weight, C...","Change from Baseline in Body Mass Index (BMI),...","Phoenix - United States, Phoenix - United Stat...",2026-04-15,2023-07-03,2024-06-18,INTERVENTIONAL,PHASE3
NCT06354660,Retatrutide,"A Phase 3, Randomized, Multicenter, Double-Bli...",Eli Lilly and Company,Unknown,RECRUITING,480,2024-04-10,Diabetes Type 2,"Retatrutide, Placebo",Change from Baseline in Hemoglobin A1c (HbA1c)...,Percentage of Participants Who Achieve HbA1c <...,"Scottsdale - United States, Tucson - United St...",2026-06,2024-04-09,2024-05-28,INTERVENTIONAL,PHASE3
NCT06260722,Retatrutide,"A Phase 3, Randomized, Multicenter, Open-label...",Eli Lilly and Company,Unknown,RECRUITING,1250,2024-02-21,"Diabetes Mellitus, Type 2","Retatrutide, Semaglutide",Change from Baseline in Hemoglobin A1c (HbA1c)...,"Change from Baseline in HbA1c (%), Percentage ...","Birmingham - United States, Escondido - United...",2026-12-11,2024-02-15,2024-06-20,INTERVENTIONAL,PHASE3
NCT05931367,Retatrutide,A Phase 3 Study to Investigate the Efficacy an...,Eli Lilly and Company,TRIUMPH-4,RECRUITING,405,2023-08-01,"Obesity, Overweight, Osteo Arthritis Knee","Retatrutide, Placebo",Change from Baseline in the Western Ontario an...,Change from Baseline in the WOMAC Physical Fun...,"Phoenix - United States, San Diego - United St...",2026-02-06,2023-07-05,2024-06-18,INTERVENTIONAL,PHASE3
NCT06297603,Retatrutide,"A Phase 3, Randomized, Multicenter, Double-Bli...",Eli Lilly and Company,Unknown,RECRUITING,320,2024-03-15,Type 2 Diabetes,"Retatrutide, Placebo",Change from Baseline in Hemoglobin A1c (HbA1c)...,Percentage of Participants Achieving HbA1c <7....,"Sun City West - United States, La Mesa - Unite...",2026-09-11,2024-03-07,2024-06-20,INTERVENTIONAL,PHASE3
NCT06383390,Retatrutide,"A Phase 3, Randomized, Double-Blind, Placebo-C...",Eli Lilly and Company,Unknown,RECRUITING,10000,2024-04-30,Overweight and Obesity,"Retatrutide, Placebo",Time to First Occurrence of Composite Endpoint...,Time to First Occurrence of Composite Endpoint...,"Birmingham - United States, Daphne - United St...",2029-02,2024-04-25,2024-06-26,INTERVENTIONAL,PHASE3
NCT05929079,Retatrutide,A Master Protocol to Investigate the Efficacy ...,Eli Lilly and Company,TRIUMPH-2,RECRUITING,1000,2023-07-11,"Type 2 Diabetes, Obesity, Overweight, Obstruct...","Retatrutide, Placebo","Percent Change from Baseline in Body Weight, C...","Change from Baseline in Body Mass Index (BMI),...","Anniston - United States, Cullman - United Sta...",2026-05-03,2023-07-03,2024-06-18,INTERVENTIONAL,PHASE3
NCT05882045,Retatrutide,"A Randomized, Double-Blind, Phase 3 Study to I...",Eli Lilly and Company,TRIUMPH-3,RECRUITING,1800,2023-05-30,"Obesity, Cardiovascular Diseases","Retatrutide, Placebo",Percent Change from Baseline in Body Weight,"Change from Baseline in Body Mass Index (BMI),...","Phoenix - United States, Covina - United State...",2026-01-20,2023-05-31,2024-06-21,INTERVENTIONAL,PHASE3
NCT05936151,Retatrutide,"A Phase 2b, Double-Blind Study to Investigate ...",Eli Lilly and Company,Unknown,RECRUITING,120,2023-07-20,"Overweight or Obesity, CKD, Type 2 Diabetes","LY3437943, Placebo",Change from Baseline in Glomerular Filtration ...,Change from Baseline in Urine Albumin-to-Creat...,"Mesa - United States, Scottsdale - United Stat...",2025-11-25,2023-07-07,2024-06-24,INTERVENTIONAL,PHASE2



Deleted rows:



,Asset,Title,Lead Sponsor,Acronym,Overall Status,Enrollment,Start Date,Conditions,Interventions,Primary Outcome(s),Secondary Outcome(s),Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases
NCT ID,,,,,,,,,,,,,,,,,



Modified rows:



,Asset,Title,Lead Sponsor,Acronym,Overall Status,Enrollment,Start Date,Conditions,Interventions,Primary Outcome(s),Secondary Outcome(s),Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases
NCT ID,,,,,,,,,,,,,,,,,
NCT01636492,Bitopertin,"Single Oral Ascending Dose Study of Safety, To...",Hoffmann-La Roche,Unknown,COMPLETED,71,2005-11,Healthy Volunteer,"bitopertin, placebo",Safety: Incidence of adverse events,Pharmacokinetics: Area under the concentration...,Christchurch - New Zealand,2006-08,2012-07-10,2016-11-02,INTERVENTIONAL,PHASE1
NCT05308472,Bitopertin,"(AURORA) A Randomized, Double-blind, Placebo-C...","Disc Medicine, Inc",Unknown,ACTIVE_NOT_RECRUITING,75,2022-10-31,Erythropoietic Protoporphyria,"DISC-1459, DISC-1459, Placebo, DISC-1459",Percent change from baseline in whole blood me...,Total hours of sunlight exposure to skin on da...,"Birmingham - United States, San Francisco - Un...",2024-02-20,2022-04-04,2024-04-15,INTERVENTIONAL,PHASE2
NCT01192880,Bitopertin,"A Phase III, Multi-Center, Randomized, 24 Week...",Hoffmann-La Roche,Unknown,COMPLETED,625,2010-11,Schizophrenia,"Placebo, Bitopertin, Antipsychotics",Mean Change From Baseline in the Positive and ...,Mean Change from Baseline in the Personal and ...,"Costa Mesa - United States, Escondido - United...",2014-07,2010-09-01,2017-02-15,INTERVENTIONAL,PHASE3
NCT05883748,Bitopertin,"An Open-Label, Long-Term Study to Investigate ...","Disc Medicine, Inc",Unknown,ENROLLING_BY_INVITATION,200,2023-08-31,Erythropoietic Protoporphyria,"DISC-1459, DISC-1459",Incidence of treatment-emergent adverse events...,Change from baseline in daily daylight toleran...,"Birmingham - United States, San Francisco - Un...",2028-06,2023-06-01,2024-02-22,INTERVENTIONAL,"PHASE2, PHASE3"
NCT02019290,Bitopertin,"A SINGLE CENTER, OPEN LABEL, FIXED SEQUENCE ST...",Hoffmann-La Roche,Unknown,WITHDRAWN,0,2014-02-10,Healthy Volunteer,"Midazolam, bitopertin",Change in plasma area under the concentration-...,Change in 1'-hydroxymidazolam plasma AUC after...,No locations listed,2014-03-31,2013-12-24,2017-10-25,INTERVENTIONAL,PHASE1
NCT05828108,Bitopertin,"A Phase I/II, Intra-Patient Dose-Escalation St...","National Heart, Lung, and Blood Institute (NHLBI)",Unknown,RECRUITING,30,2023-07-25,Steroid-refractory Diamond-Blackfan Anemia (DBA),Bitopertin,Rate of response,"Rate of relapse, Long-term safety of drug, Tol...",Bethesda - United States,2029-05-01,2023-04-25,2024-04-10,INTERVENTIONAL,"PHASE1, PHASE2"
NCT03271541,Bitopertin,"A Phase II, Single Arm, Multicenter, Proof-of-...",Hoffmann-La Roche,Unknown,COMPLETED,12,2017-10-26,Beta-Thalassemia,Bitopertin,Safety Outcome: Percentage of Participants wit...,"Apparent Clearance of Bitopertin, Volume of Di...","Genova - Italy, Milano - Italy, Baabda - Leban...",2018-06-29,2017-09-05,2018-10-05,INTERVENTIONAL,PHASE2
NCT01192906,Bitopertin,"A Phase III, Multi-center, Randomized, 24 Week...",Hoffmann-La Roche,Unknown,COMPLETED,624,2010-12,Schizophrenia,"Placebo, bitopertin [RO4917838], bitopertin [R...",Efficacy (PANSS negative symptoms factor score...,No secondary outcomes listed,"Carson - United States, Oceanside - United Sta...",2014-07,2010-09-01,2016-11-02,INTERVENTIONAL,PHASE3
NCT01234779,Bitopertin,"A Phase II, Multi-center, Randomized, 4-week, ...",Hoffmann-La Roche,Unknown,COMPLETED,301,2011-02,Schizophrenia,"bitopertin [RO4917838], bitopertin [RO4917838]...",Change in Positive and Negative Syndrome Scale...,"Clinical response, defined as at least 30% or ...","Little Rock - United States, Cerritos - United...",2012-09,2010-11-04,2016-11-02,INTERVENTIONAL,PHASE2
